In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score


df = pd.read_csv('Final Transactions.csv')


df = df.drop(columns=['TRANSACTION_ID', 'TX_DATETIME', 'CUSTOMER_ID', 'TERMINAL_ID', 'TX_FRAUD_SCENARIO'])


df.fillna(df.median(), inplace=True)


X = df.drop(columns=['TX_FRAUD'])
y = df['TX_FRAUD']


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


input_dim = X_train.shape[1]

input_layer = Input(shape=(input_dim,))
encoded = Dense(16, activation="relu")(input_layer)
encoded = Dense(8, activation="relu")(encoded)

decoded = Dense(16, activation="relu")(encoded)
decoded = Dense(input_dim, activation="sigmoid")(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')


X_train_normal = X_train[y_train == 0]  # Use only non-fraudulent transactions
autoencoder.fit(X_train_normal, X_train_normal, epochs=20, batch_size=32, validation_data=(X_test, X_test))

X_test_pred = autoencoder.predict(X_test)
reconstruction_error = np.mean(np.abs(X_test - X_test_pred), axis=1)


threshold = np.percentile(reconstruction_error, 95)  # 95th percentile


y_pred = (reconstruction_error > threshold).astype(int)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print("Autoencoder Testing Accuracy:", accuracy)
print("Autoencoder Precision:", precision)


Epoch 1/20
33223/33223 ━━━━━━━━━━━━━━━━━━━━ 144s 4ms/step - loss: 0.4331 - val_loss: 0.6242
Epoch 2/20
33223/33223 ━━━━━━━━━━━━━━━━━━━━ 150s 4ms/step - loss: 0.4174 - val_loss: 0.6251
Epoch 3/20
33223/33223 ━━━━━━━━━━━━━━━━━━━━ 138s 4ms/step - loss: 0.4185 - val_loss: 0.6249
Epoch 4/20
33223/33223 ━━━━━━━━━━━━━━━━━━━━ 140s 4ms/step - loss: 0.4178 - val_loss: 0.6250
Epoch 5/20
33223/33223 ━━━━━━━━━━━━━━━━━━━━ 132s 4ms/step - loss: 0.4188 - val_loss: 0.6257
Epoch 6/20
33223/33223 ━━━━━━━━━━━━━━━━━━━━ 142s 4ms/step - loss: 0.4186 - val_loss: 0.6259
Epoch 7/20
33223/33223 ━━━━━━━━━━━━━━━━━━━━ 163s 5ms/step - loss: 0.4182 - val_loss: 0.6265
Epoch 8/20
33223/33223 ━━━━━━━━━━━━━━━━━━━━ 152s 5ms/step - loss: 0.4178 - val_loss: 0.6267
Epoch 9/20
33223/33223 ━━━━━━━━━━━━━━━━━━━━ 139s 4ms/step - loss: 0.4182 - val_loss: 0.6267
Epoch 10/20
33223/33223 ━━━━━━━━━━━━━━━━━━━━ 138s 4ms/step - loss: 0.4181 - val_loss: 0.6269
Epoch 11/20
33223/33223 ━━━━━━━━━━━━━━━━━━━━ 146s 4ms/step - loss: 0.4185 - val